Variables to change:

1. jar_file_loc: this is the file of the .jar postgres file. Change path to where it is located
2. postgres credentials (username, password, url): The default is linked to a PostgreSQL instance created in my GCP account. However, once I disable biling, it will be suspsended. I will be disabling billing to avoid wasting credits.
3. train_data_path & test_data_path: change to path of train/test csv files

In [2]:
# install if necessary 
# AFTER INSTALLATION: restart Kernel

!pip install tensorflow==2.9.2
!pip install numpy==1.21.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 94.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 107.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 79.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 9.6 MB/s eta 

In [1]:
import tensorflow as tf  # now import the tensorflow module
print(tf.__version__)  # make sure the version is 2.x

2022-11-28 21:29:04.819349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 21:29:04.819394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.2


# Task I

In [1]:
jar_file_loc = 'gs://dataproc-staging-northamerica-northeast2-1049722977636-el8of1h1/postgresql-42.5.0.jar'

# set up Spark
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .config('spark.jars', jar_file_loc) \
    .getOrCreate()


#Access SparkContext from your SparkSession
print("APP Name :"+ spark.sparkContext.appName);
print("Master :"+ spark.sparkContext.master);

sqlContext = SQLContext(spark.sparkContext)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/28 21:30:57 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/11/28 21:30:57 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/11/28 21:30:57 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/11/28 21:30:57 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


APP Name :GenericAppName
Master :local[*]


In [2]:
# read in data

train_data_path = 'gs://dataproc-staging-northamerica-northeast2-1049722977636-el8of1h1/train70_reduced.csv'
test_data_path = 'gs://dataproc-staging-northamerica-northeast2-1049722977636-el8of1h1/test30_reduced.csv'

df_train = spark.read.csv(train_data_path, header = True, inferSchema = True)
df_test = spark.read.csv(test_data_path, header = True, inferSchema = True)

In [3]:
# add column to differentiate b/w train and test sets
from pyspark.sql.functions import col, lit

df_train_cat = df_train.withColumn("data_category", lit("train"))
df_test_cat = df_test.withColumn("data_category", lit("test"))

print('Item Count\n')
print('Train:', df_train_cat.count())
print('Test:', df_test_cat.count())


# combine dfs
df_combined = df_train_cat.union(df_test_cat)
print('Combined:', df_combined.count())

Item Count

Train: 231646
Test: 99290
Combined: 330936


In [5]:
# write into postgresql db

db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="bigdata"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://34.130.206.1/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "mqtt"

# create df with train data 
df_combined.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", "bigdata")\
.option("Driver", db_properties['driver'])\
.save()

In [6]:
# read db to ensure data has been written in correctly
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", "bigdata")\
    .option("Driver", db_properties['driver'])\
    .load()

print('Item Count from PostgreSQL Read:', df_read.count())

Item Count from PostgreSQL Read: 330936


# Task III

### Data Processing

In [7]:
# replacing all . with _ in column names to avoid bugs with having . in col name

col_names_underscore = ['tcp_flags','tcp_time_delta','tcp_len','mqtt_conack_flags','mqtt_conack_flags_reserved',
             'mqtt_conack_flags_sp','mqtt_conack_val','mqtt_conflag_cleansess','mqtt_conflag_passwd',
             'mqtt_conflag_qos','mqtt_conflag_reserved','mqtt_conflag_retain','mqtt_conflag_uname',
             'mqtt_conflag_willflag','mqtt_conflags','mqtt_dupflag','mqtt_hdrflags','mqtt_kalive',
             'mqtt_len','mqtt_msg','mqtt_msgid','mqtt_msgtype','mqtt_proto_len','mqtt_protoname',
             'mqtt_qos','mqtt_retain','mqtt_sub_qos','mqtt_suback_qos','mqtt_ver','mqtt_willmsg',
             'mqtt_willmsg_len','mqtt_willtopic','mqtt_willtopic_len','target','data_category']

df_read_underscore = df_read.toDF(*col_names_underscore)

In [8]:
# remove all zero columns & absurd cols

zero_cols = ['mqtt_conack_flags','mqtt_conack_flags_reserved','mqtt_conack_flags_sp','mqtt_conack_val',
            'mqtt_conflag_qos','mqtt_conflag_reserved','mqtt_conflag_retain','mqtt_conflag_willflag',
            'mqtt_retain','mqtt_sub_qos','mqtt_suback_qos','mqtt_willmsg',
             'mqtt_willmsg_len','mqtt_willtopic','mqtt_willtopic_len']

absurd_cols = ['mqtt_msg', 'mqtt_msgid']


df_read_underscore_dropped = df_read_underscore.drop(*zero_cols+absurd_cols)

In [9]:
# split train and test
df_read_train = df_read_underscore_dropped.where(df_read_underscore_dropped['data_category'] == 'train')
df_read_test = df_read_underscore_dropped.where(df_read_underscore_dropped['data_category'] == 'test')


# drop 'data_category' col
df_read_train = df_read_train.drop('data_category')
df_read_test = df_read_test.drop('data_category')

In [10]:
# PySpark
import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression

from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np


In [11]:
col_names_underscore = ['tcp_flags','tcp_time_delta','tcp_len','mqtt_conack_flags','mqtt_conack_flags_reserved',
             'mqtt_conack_flags_sp','mqtt_conack_val','mqtt_conflag_cleansess','mqtt_conflag_passwd',
             'mqtt_conflag_qos','mqtt_conflag_reserved','mqtt_conflag_retain','mqtt_conflag_uname',
             'mqtt_conflag_willflag','mqtt_conflags','mqtt_dupflag','mqtt_hdrflags','mqtt_kalive',
             'mqtt_len','mqtt_msg','mqtt_msgid','mqtt_msgtype','mqtt_proto_len','mqtt_protoname',
             'mqtt_qos','mqtt_retain','mqtt_sub_qos','mqtt_suback_qos','mqtt_ver','mqtt_willmsg',
             'mqtt_willmsg_len','mqtt_willtopic','mqtt_willtopic_len','target'] # 'data_category' removed


nominal_cols = ['tcp_flags','mqtt_conflags','mqtt_hdrflags', 'mqtt_protoname']

continuous_cols = ['tcp_time_delta', 'tcp_len','mqtt_kalive','mqtt_len',  'mqtt_msgtype',
                  'mqtt_proto_len','mqtt_qos', 'mqtt_ver']

binary_cols = ['mqtt_conflag_cleansess', 'mqtt_conflag_passwd','mqtt_conflag_uname', 'mqtt_dupflag']


keys = ['slowite', 'bruteforce','flood', 'malformed', 'dos', 'legitimate']
vals = [0, 1, 2, 3, 4, 5]

global label_dict
label_dict = dict(zip(keys, vals))

# ===========================================================================

class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        label_to_multiclass = udf(lambda name: label_dict[name])
        output_df = dataset.withColumn('outcome', label_to_multiclass(col('target'))).drop('target')
        output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))

        return output_df
    
class ColumnDropper(Transformer): # this transformer drops uannecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )
    
    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)
    
    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+binary_cols+nominal_onehot_cols
    corelated_cols_to_remove = ['mqtt_conflag_uname','mqtt_qos','mqtt_proto_len', 'mqtt_ver']

    for col_name in corelated_cols_to_remove:
        feature_cols.remove(col_name)
    
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columns, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_cols+nominal_id_cols+
        nominal_onehot_cols+ binary_cols + continuous_cols + ['vectorized_features'])
     
        
    # fit with logistic regression
    lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome', maxIter=10)
    
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    return pipeline

In [12]:
# fit and transform
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(df_read_train)

train_transform = preprocess_pipeline_model.transform(df_read_train)
test_transform = preprocess_pipeline_model.transform(df_read_test)

### Machine Learning - PySpark

The classifiers chosen are: Logistic Regression & Random Forest.

#### Standard Training

In [13]:
# logistic regression

lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome', maxIter=10)
lr_fit = lr.fit(train_transform)

lr_preds_train = lr_fit.transform(train_transform)
lr_preds_test = lr_fit.transform(test_transform)

# random forest
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'outcome')
rf_fit = rf.fit(train_transform)

rf_preds_train = rf_fit.transform(train_transform)
rf_preds_test = rf_fit.transform(test_transform)

22/11/28 21:32:13 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/28 21:32:13 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
Traceback (most recent call last):                                              
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [14]:
# accuracies

# logistic regression
lr_accuracy_train = (lr_preds_train.filter(lr_preds_train.outcome == lr_preds_train.prediction)
    .count() / float(lr_preds_train.count()))

lr_accuracy_test = (lr_preds_test.filter(lr_preds_test.outcome == lr_preds_test.prediction)
    .count() / float(lr_preds_test.count()))

print('Logistic Regression')
print("Train Accuracy :", lr_accuracy_train)
print("Test Accuracy :", lr_accuracy_test)


# rf
rf_accuracy_train = (rf_preds_train.filter(rf_preds_train.outcome == rf_preds_train.prediction)
    .count() / float(rf_preds_train.count()))

rf_accuracy_test = (rf_preds_test.filter(rf_preds_test.outcome == rf_preds_test.prediction)
    .count() / float(rf_preds_test.count()))

print('\nRandom Forest')
print("Train Accuracy :", rf_accuracy_train)
print("Test Accuracy :", rf_accuracy_test)

Logistic Regression
Train Accuracy : 0.8226690726366956
Test Accuracy : 0.8217443851344546



Random Forest
Train Accuracy : 0.8468266233822298
Test Accuracy : 0.8588881055494008


#### Hyperparameter Tuning

In [15]:
# logistic regression

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

lr = LogisticRegression(featuresCol = 'features', labelCol = 'outcome')

# Create ParamGrid for Cross Validation
lr_paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0001, 1.0])
             .addGrid(lr.maxIter, [10, 50])
             .build())

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', 
    labelCol='outcome', metricName='accuracy')

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=3)

lr_cv_fit_train = lr_cv.fit(train_transform)
lr_cv_preds_test = lr_cv_fit_train.transform(test_transform)



In [16]:
# random forest

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'outcome')

rf_paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [10, 15])# maximum depth for each tree
             .addGrid(rf.numTrees,[30, 60])# number of trues
             .build())

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', 
    labelCol='outcome', metricName='accuracy')

rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, 
                    evaluator=evaluator, numFolds=3)

rf_cv_fit_train = rf_cv.fit(train_transform)

rf_cv_preds_test = rf_cv_fit_train.transform(test_transform)

Traceback (most recent call last):                                              
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [17]:
# logistic regression
lr_accuracy_test_cv = (lr_cv_preds_test.filter(lr_cv_preds_test.outcome == lr_cv_preds_test.prediction)
    .count() / float(lr_cv_preds_test.count()))

# random forest
rf_accuracy_test_cv = (rf_cv_preds_test.filter(rf_cv_preds_test.outcome == rf_cv_preds_test.prediction)
    .count() / float(rf_cv_preds_test.count()))

print('\nLogistic Regression')
print("Pre-CV:", lr_accuracy_test)
print("Post-CV:", lr_accuracy_test_cv)


print("\nRandom Forest")
print("Pre-CV:", rf_accuracy_test)
print("Post-CV:", rf_accuracy_test_cv)



Logistic Regression
Pre-CV: 0.8217443851344546
Post-CV: 0.8266391378789405

Random Forest
Pre-CV: 0.8588881055494008
Post-CV: 0.8970188337194078


### Machine Learning - TensorFlow

The two classifiers chosen are: a shallow NN and a deep NN. The shallow NN only has 2 hidden layers, while the deep NN has 5 hidden layers.

In [18]:
import tensorflow as tf  # now import the tensorflow module
print(tf.__version__)  # make sure the version is 2.x

2022-11-28 21:40:55.998412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-28 21:40:55.998451: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.2


In [19]:
import numpy as np
from tensorflow import keras

In [20]:
# create tensors

x_train = tf.constant(np.array(train_transform.toPandas()['features'].values.tolist()))
y_train = tf.constant(np.array(train_transform.toPandas()['outcome'].values.tolist()))

x_test = tf.constant(np.array(test_transform.toPandas()['features'].values.tolist()))
y_test = tf.constant(np.array(test_transform.toPandas()['outcome'].values.tolist()))

2022-11-28 21:41:54.069861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-28 21:41:54.069912: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-28 21:41:54.069941: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default-50gbram-newnew-m): /proc/driver/nvidia/version does not exist
2022-11-28 21:41:54.070332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:
# Shallow NN (2 hidden layers)
from tensorflow import keras

model_shallow = keras.Sequential()

model_shallow.add(keras.layers.Dense(10, activation = 'relu'))
model_shallow.add(keras.layers.Dense(10, activation = 'relu'))

model_shallow.add(keras.layers.Dense(6))

model_shallow.compile(optimizer = keras.optimizers.SGD(), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model_shallow.fit(x_train, y_train, epochs = 10, verbose = 2)
model_shallow.evaluate(x_test, y_test, verbose = 2)

Epoch 1/10
7239/7239 - 8s - loss: 0.5159 - sparse_categorical_accuracy: 0.7947 - 8s/epoch - 1ms/step
Epoch 2/10
7239/7239 - 7s - loss: 0.4390 - sparse_categorical_accuracy: 0.8128 - 7s/epoch - 1ms/step
Epoch 3/10
7239/7239 - 7s - loss: 0.4324 - sparse_categorical_accuracy: 0.8185 - 7s/epoch - 1ms/step
Epoch 4/10
7239/7239 - 7s - loss: 0.4292 - sparse_categorical_accuracy: 0.8195 - 7s/epoch - 999us/step
Epoch 5/10
7239/7239 - 7s - loss: 0.4274 - sparse_categorical_accuracy: 0.8210 - 7s/epoch - 999us/step
Epoch 6/10
7239/7239 - 7s - loss: 0.4263 - sparse_categorical_accuracy: 0.8222 - 7s/epoch - 1000us/step
Epoch 7/10
7239/7239 - 7s - loss: 0.4256 - sparse_categorical_accuracy: 0.8220 - 7s/epoch - 1ms/step
Epoch 8/10
7239/7239 - 7s - loss: 0.4249 - sparse_categorical_accuracy: 0.8226 - 7s/epoch - 1ms/step
Epoch 9/10
7239/7239 - 7s - loss: 0.4245 - sparse_categorical_accuracy: 0.8228 - 7s/epoch - 1ms/step
Epoch 10/10
7239/7239 - 7s - loss: 0.4240 - sparse_categorical_accuracy: 0.8235 - 7s

[0.43267548084259033, 0.820636510848999]

In [22]:
from tensorflow import keras

model_deep = keras.Sequential()

model_deep.add(keras.layers.Dense(10, activation = 'relu'))
model_deep.add(keras.layers.Dense(10, activation = 'relu'))
model_deep.add(keras.layers.Dense(10, activation = 'relu'))
model_deep.add(keras.layers.Dense(10, activation = 'relu'))
model_deep.add(keras.layers.Dense(10, activation = 'relu'))


model_deep.add(keras.layers.Dense(6))

model_deep.compile(optimizer = keras.optimizers.SGD(), loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model_deep.fit(x_train, y_train, epochs = 10, verbose = 2)
model_deep.evaluate(x_test, y_test, verbose = 2)

Epoch 1/10
7239/7239 - 8s - loss: 0.5618 - sparse_categorical_accuracy: 0.7683 - 8s/epoch - 1ms/step
Epoch 2/10
7239/7239 - 8s - loss: 0.4388 - sparse_categorical_accuracy: 0.8202 - 8s/epoch - 1ms/step
Epoch 3/10
7239/7239 - 8s - loss: 0.4310 - sparse_categorical_accuracy: 0.8242 - 8s/epoch - 1ms/step
Epoch 4/10
7239/7239 - 8s - loss: 0.4292 - sparse_categorical_accuracy: 0.8243 - 8s/epoch - 1ms/step
Epoch 5/10
7239/7239 - 8s - loss: 0.4277 - sparse_categorical_accuracy: 0.8247 - 8s/epoch - 1ms/step
Epoch 6/10
7239/7239 - 8s - loss: 0.4286 - sparse_categorical_accuracy: 0.8239 - 8s/epoch - 1ms/step
Epoch 7/10
7239/7239 - 8s - loss: 0.4253 - sparse_categorical_accuracy: 0.8249 - 8s/epoch - 1ms/step
Epoch 8/10
7239/7239 - 8s - loss: 0.4261 - sparse_categorical_accuracy: 0.8241 - 8s/epoch - 1ms/step
Epoch 9/10
7239/7239 - 8s - loss: 0.4245 - sparse_categorical_accuracy: 0.8247 - 8s/epoch - 1ms/step
Epoch 10/10
7239/7239 - 8s - loss: 0.4239 - sparse_categorical_accuracy: 0.8248 - 8s/epoch 

[0.43518850207328796, 0.8196696639060974]

#### Hyperparameter Tuning

In [24]:
# combine the x and y train tensors and shuffle
tf_train = tf.concat([x_train, tf.reshape(y_train, [-1, 1])], axis = 1)
tf_train_shuffle = tf.random.shuffle(tf_train)

In [25]:
def shallow_cross_val_activation_width(k, act_fun, width):
    
    cuts = np.linspace(0, tf_train_shuffle.shape[0]-1, k+1, dtype = int)
    
    model = keras.Sequential()
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(6))

    metric = []

    for i in range(k):
        print('k = '+str(i+1)+'\n')
        
        val = tf_train_shuffle[cuts[i]:cuts[i+1]]
            
        mask = np.ones(tf_train_shuffle.shape[0])
        mask[cuts[i]:cuts[i+1]] = 0
        
        train = tf.boolean_mask(tf_train_shuffle, mask)
    
        cur_x_train = train[:,:-1]
        cur_y_train = train[:,-1]
        
        cur_x_val = val[:,:-1]
        cur_y_val = val[:,-1]
        
        model.compile(optimizer = keras.optimizers.SGD(), 
                      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[keras.metrics.SparseCategoricalAccuracy()])
        
        fit_data = model.fit(cur_x_train, cur_y_train, epochs = 5, 
                             verbose = 2,validation_data = (cur_x_val, cur_y_val))
        
        cur_auc = np.mean(fit_data.history['val_sparse_categorical_accuracy'])
        metric.append(cur_auc)
        
        if np.max(metric) == cur_auc:
            best_model = model
            print('\nNew best model saved.')
        
        print('\n')
        
    print(metric)
    return np.mean(metric), best_model.evaluate(x_test, y_test, verbose = 2), best_model


# val_acc, test_res, best_model_shallow = shallow_cross_val_activation_width(k = 4, act_fun = 'relu', width = 5)

# print('\n===========')
# print("Validation Accuracy:", val_acc)
# print("Test Accuracy:", test_res[1])

In [28]:
def deep_cross_val_activation_width(k, act_fun, width):
    
    cuts = np.linspace(0, tf_train_shuffle.shape[0]-1, k+1, dtype = int)
    
    model = keras.Sequential()
    
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(width, activation = act_fun))
    model.add(keras.layers.Dense(width, activation = act_fun))
    
    model.add(keras.layers.Dense(6))

    metric = []

    for i in range(k):
        print('k = '+str(i+1)+'\n')
        
        val = tf_train_shuffle[cuts[i]:cuts[i+1]]
            
        mask = np.ones(tf_train_shuffle.shape[0])
        mask[cuts[i]:cuts[i+1]] = 0
        
        train = tf.boolean_mask(tf_train_shuffle, mask)
    
        cur_x_train = train[:,:-1]
        cur_y_train = train[:,-1]
        
        cur_x_val = val[:,:-1]
        cur_y_val = val[:,-1]
        
        model.compile(optimizer = keras.optimizers.SGD(), 
                      loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=[keras.metrics.SparseCategoricalAccuracy()])
        
        fit_data = model.fit(cur_x_train, cur_y_train, epochs = 5,
                             verbose = 2, validation_data = (cur_x_val, cur_y_val))
        
        cur_auc = np.mean(fit_data.history['val_sparse_categorical_accuracy'])
        metric.append(cur_auc)
        
                
        if np.max(metric) == cur_auc:
            best_model = model
            print('\nNew best model saved.')

        print('\n')
        
    print(metric)
    return np.mean(metric), best_model.evaluate(x_train, y_train, verbose = 2), best_model


# val_acc, test_res, best_model_deep = deep_cross_val_activation_width(k = 4, act_fun = 'relu', width = 5)

# print('\n===========')
# print("Validation Accuracy:", val_acc)
# print("Test Accuracy:", test_res[1])

In [26]:
act_funs = ['relu', 'softmax']
widths = [10, 20]

accuracies = []

for act_fun in act_funs:
    for width in widths:

        run_name = 'run-'+act_fun+' width'+str(width)
        print('')
        print('--- Starting trial: %s' % run_name)

        run_dir = 'logs14813/hparam_tuning_q2_2/' + run_name
        val_acc, test_res, hp_model = shallow_cross_val_activation_width(3, act_fun, width)
        
        accuracies.append(test_res[1])
        
        if np.max(accuracies) == test_res[1]:
            best_hp_shallow_model = hp_model
            best_shallow_run_name = run_name
            print('New Best Model:', best_shallow_run_name)
            
best_hp_shallow_model.evaluate(x_test, y_test, verbose = 2)
shallow_res = best_hp_shallow_model.evaluate(x_test, y_test, verbose = 2)

print('Best Model Hyperparameters:', best_shallow_run_name)
print('Test Accuracy:', shallow_res[1])


--- Starting trial: run-relu width10
k = 1

Epoch 1/5
4826/4826 - 8s - loss: 0.5326 - sparse_categorical_accuracy: 0.7962 - val_loss: 0.4558 - val_sparse_categorical_accuracy: 0.8188 - 8s/epoch - 2ms/step
Epoch 2/5
4826/4826 - 7s - loss: 0.4435 - sparse_categorical_accuracy: 0.8160 - val_loss: 0.4481 - val_sparse_categorical_accuracy: 0.8214 - 7s/epoch - 1ms/step
Epoch 3/5
4826/4826 - 7s - loss: 0.4345 - sparse_categorical_accuracy: 0.8194 - val_loss: 0.4478 - val_sparse_categorical_accuracy: 0.8227 - 7s/epoch - 1ms/step
Epoch 4/5
4826/4826 - 7s - loss: 0.4294 - sparse_categorical_accuracy: 0.8224 - val_loss: 0.4342 - val_sparse_categorical_accuracy: 0.8235 - 7s/epoch - 1ms/step
Epoch 5/5
4826/4826 - 7s - loss: 0.4256 - sparse_categorical_accuracy: 0.8236 - val_loss: 0.4333 - val_sparse_categorical_accuracy: 0.8204 - 7s/epoch - 1ms/step

New best model saved.


k = 2

Epoch 1/5
4826/4826 - 7s - loss: 0.4298 - sparse_categorical_accuracy: 0.8229 - val_loss: 0.4274 - val_sparse_categori

In [29]:
act_funs = ['relu', 'softmax']
widths = [10, 20]

accuracies = []

for act_fun in act_funs:
    for width in widths:

        run_name = 'run-'+act_fun+' width'+str(width)
        print('')
        print('--- Starting trial: %s' % run_name)
        
        val_acc, test_res, hp_model = deep_cross_val_activation_width(3, act_fun, width)
        
        accuracies.append(test_res[1])
        
        if np.max(accuracies) == test_res[1]:
            best_hp_deep_model = hp_model
            best_deep_run_name = run_name
            print('New Best Model:', best_deep_run_name)


deep_res = best_hp_deep_model.evaluate(x_test, y_test, verbose = 2)

print('Best Model Hyperparameters:', best_deep_run_name)
print('Test Accuracy:', deep_res[1])


--- Starting trial: run-relu width10
k = 1

Epoch 1/5
4826/4826 - 8s - loss: 0.5444 - sparse_categorical_accuracy: 0.7887 - val_loss: 0.4542 - val_sparse_categorical_accuracy: 0.8154 - 8s/epoch - 2ms/step
Epoch 2/5
4826/4826 - 8s - loss: 0.4361 - sparse_categorical_accuracy: 0.8226 - val_loss: 0.4410 - val_sparse_categorical_accuracy: 0.8227 - 8s/epoch - 2ms/step
Epoch 3/5
4826/4826 - 8s - loss: 0.4265 - sparse_categorical_accuracy: 0.8253 - val_loss: 0.4345 - val_sparse_categorical_accuracy: 0.8214 - 8s/epoch - 2ms/step
Epoch 4/5
4826/4826 - 8s - loss: 0.4237 - sparse_categorical_accuracy: 0.8257 - val_loss: 0.4389 - val_sparse_categorical_accuracy: 0.8227 - 8s/epoch - 2ms/step
Epoch 5/5
4826/4826 - 8s - loss: 0.4224 - sparse_categorical_accuracy: 0.8256 - val_loss: 0.4357 - val_sparse_categorical_accuracy: 0.8214 - 8s/epoch - 2ms/step

New best model saved.


k = 2

Epoch 1/5
4826/4826 - 8s - loss: 0.4268 - sparse_categorical_accuracy: 0.8241 - val_loss: 0.4208 - val_sparse_categori